# 8. 컴퓨터 비전을 위한 딥러닝

---

## 8.1 합성곱 신경망 소개

## 8.2 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기

## 8.3 사전 훈련된 모델 활용하기

## 8.4 요약

---

## 8.1 합성곱 신경망 소개

MNIST 데이터셋을 사용한 합성곱 신경망 (컨브넷) 예제를 확인해보자.

In [1]:
## 간단한 컨브넷 만들기
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = x = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

컨브넷은 배치 차원을 제외하고 `(image_height, image_width, image_channels)` 크기의 입력 텐서를 사용한다.

In [2]:
## 컨브넷의 구조 출력
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856 

In [3]:
## MNIST 이미지에서 컨브넷 훈련하기
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# dataset amount is 60 thousands, then 28 x 28 grayscale color
train_images = train_images.reshape((60000, 28, 28, 1))
# regularization
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
# regularization
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics = ["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=4)

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/5
15000/15000 [==============================] - 119s 8ms/step - loss: 0.1061 - accuracy: 0.9689
Epoch 2/5
15000/15000 [==============================] - 117s 8ms/step - loss: 0.0552 - accuracy: 0.9867
Epoch 3/5
15000/15000 [==============================] - 117s 8ms/step - loss: 0.0434 - accuracy: 0.9903
Epoch 4/5
15000/15000 [==============================] - 118s 8ms/step - loss: 0.0364 - accuracy: 0.9925
Epoch 5/5
15000/15000 [==============================] - 121s 8ms/step - loss: 0.0343 - accuracy: 0.9933


In [4]:
## 컨브넷 평가하기
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"테스트 정확도: {test_acc:.3f}")

313/313 [==============================] - 3s 10ms/step - loss: 0.0480 - accuracy: 0.9905
테스트 정확도: 0.990


완전 연결된 모델(`Dense() | fully connected model`)보다 간단한 커브넷 모델의 테스트 정확도가 더 높음이 확인되었다.

---

### 8.1.1 합성곱 연산

&nbsp;</br>
완전 연결 층(`fully connected layer`)와 합성곱 층(`convolutional layer`)의 근본적인 차이</br>
1. **fully connected layer**
- 입력 특성 공간(input feature space)에 있는 전역 패턴을 학습
  - 예시: MINIST의 경우 숫자 이미지에서 모든 픽셀에 대해 패턴을 학습함
2. **convolutional layer**
- 합성곱 층은 지역 패턴을 학습함
  - 예시: 이미지일 경우 작은 2D 윈도우(window)로 입력해서 패턴을 찾음

---

![이미지는 에지(edge), 질감(texture) 등 지역 패턴으로 분해될 수 있다.](./images/lec8/1.png)</br>

&nbsp;</br>
`2번`의 특징은 컨브넷에 두 가지 흥미로운 성질을 제공한다.</br>
&nbsp;</br>
1. **학습된 패턴은 평행이동 불변성**(translation invariant)**을 가진다.**
  - 컨브넷의 윈도우(a.k.a `filter`)가 이미지의 특정 위치(ex: `이미지의 오른쪽 아래 모서리`)에서 패턴을 학습했다면, 다른 곳(ex: `이미지의 왼쪽 위 모서리`)에서도 패턴을 학습할 수 있다.
    > 이러한 처리 방법은 `적은 수의 훈련 샘플을 사용해 일반화 능력을 가진 표현을 학습`할 수 있게 한다.
    >> **사람이 보는 세상은 평행 이동으로 인해 세상이 다르게 인식되지 않는다**
2. **컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다.**
  - 첫 번째 합성곱 층이 `edge` 같은 작은 지역 패턴을 학습
  - 두 번째 합성곱 층이 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습
    > 이런 방식을 통해 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있다.

**특성 맵**(feature map)</br>
- Rank-3 Tensor, 합성곱 연산이 적용되는 텐서
  - 2개의 공간 축(`높이`와 `넓이`)과 깊이 축(또는 `채널 축`)으로 구성되어 있는 텐서

합성곱 연산 과정
- 입력 특성 맵에서 작은 패치(patch)들을 추출
- 추출된 모든 패치에 같은 변환을 적용
- 변환된 패치들을 가지고 **출력 특성 맵**(output featrue map)을 생성

> patch?
>> 이미지의 픽셀, 이 때 픽셀은 `filter` 또는 `kernel`로 걸러진 이미지의 픽셀을 말한다.

![우리가 보는 세상은 시각적 구성 요소들의 공간적인 계층 구조로 구성되어 있으며, 기본적인 직선이나 질감들이 연결되어 눈이나 귀 같은 간단한 구성 요소를 만들고 이들이 모여서 "cat"처럼 고수준의 개념을 만든다.](./images/lec8/2.png)</br>

---



**필터**(filter)
- 합성곱에서 입력 데이터의 어떤 특성을 인코딩하는 것

**응답 맵**(response map)
- 입력 이미지의 각 픽셀에서 필터 패턴에 대한 응답을 나타내는 것

```shell
 Layer (type)                Output Shape              Param #   
=================================================================
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320
```

```text
이미지(28, 28, 1)를 Input할 때, keras는 default로 `same` padding과 `1` stride를 적용한다.

따라서 `input_1` 레이어의 응답 맵은 (28,28,X) 크기의 배열값을 가진다. 이 때, X는 합성곱 층의 필터 개수를 의미하는데, input 단계에서는 적용된 필터가 없어 실제로 input 단계 이후의 응답 맵은 (28, 28, 1)의 크기를 가지게 된다.
```

![응답 맵의 개념: 입력의 각 위치에서 한 패턴의 존재에 대한 2D 맵](./images/lec8/3.png)</br>

&nbsp;</br>
합성곱은 핵심적인 2개의 파라미터로 정의된다.
1. **입력으로부터 뽑아닐 패치의 크기**
  - 일반적으로 `3x3` 또는 `5x5` 크기를 사용
    - a.k.a filter_size or kernel_size
2. **특성 맵의 출력 깊이**
  - 합성곱으로 계산할 필터 개수
    - 위 예제에서는 깊이가 `32` &rarr; `128`로 수행되었다.

![합성곱 작동 방식](./images/lec8/4.png)</br>
&nbsp;</br>
두 가지 이유로 출력 높이와 너비는 입력의 높이, 너비와 다를 수 있다.</br>
&nbsp;</br>
- 경계 문제
  - 입력 특성 맵에 패딩을 추가하여 대응 가능
- **스트라이드**(stride)의 사용 여부

---

#### 경계 문제와 패딩 이해하기

입력이 `5x5` 크기의 특성 맵이고, `3x3` 크기의 filter를 사용하면 출력 특성 맵의 크기는 `3x3`이 된다.</br>

&nbsp;</br>
![5x5 입력 특성 맵에서 가능한 3x3 패치 위치](./images/lec8/5.png)</br>

&nbsp;</br>
입력과 동일한 높이, 너비를 가진 출력 특성 맵을 얻으려면 **패딩**(padding)을 사용할 수 있다.
> `padding`: 입력 특성 맵의 가장자리에 적절한 개수의 행과 열 추가
>> 추가 되는 행과 열은 `0`으로 채워진다.
>>> 이 때문에 `zero padding`이라고도 부른다.

![25개의 3x3 패치를 뽑기 위해 5x5 입력에 패딩 추가하기](./images/lec8/6.png)</br>

&nbsp;</br>
`Conv2D()`에서 패딩은 매개변수 `padding`으로 설정가능하다.
- `valid`: 패딩을 사용하지 않는다는 의미
  - 입력 특성 맵에 비해 출력 특성 맵의 크기가 작아지는 특징이 있다.
- `same`: 입력과 동일한 높이와 너비를 가진 출력을 만들기 위해 패딩한다는 의미

---

#### 합성곱 스트라이드 이해하기

**스트라이드**(stride)
- 출력 크기에 영향을 미치는 요소 중 하나
  - default is `1`
- 1보다 큰 값을 가지는 **스트라이드 합성곱**도 가능

![2x2 스트라이드를 사용한 3x3 합성곱의 패치](./images/lec8/7.png)</br>

&nbsp;</br>
스트라이드가 1보다 크면(`N`) 특성 맵의 너비와 높이가 `N의 배수`로 다운샘플링 되었다는 뜻이다.
- padding에 따라 더 줄어 들 수 있다.

> 스트라이드 합성곱은 분류 모델에서 드물게 사용됨

일반적으로 분류 모델에서는 특성 맵을 다운샘플링 하기 위해 다음 연산을 주로 사용한다.
- **최대 풀링** (max pooling)

### 8.1.2 최대 풀링 연산

최대풀링의 역할
- 강제적으로 특성 맵을 다운샘플링
  - 스트라이드 합성 곱과 비슷한 연산과정을 수행

&nbsp;</br>
최대 풀링
- 입력 특성 맵에서 윈도우(`filter`)에 맞는 패치를 추출하고 각 채널별로 최댓값을 출력

